# Lending club case study

In [36]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df = pd.read_csv("./data/loan.csv", low_memory=False)
df.shape

(39717, 111)

## Handle missing values in the data

In [38]:
dfnull = pd.DataFrame()
dfnull['nans'] = df.isnull().sum()
dfnull['nans'].value_counts()

39717    54
0        43
56        2
25682     1
71        1
36931     1
697       1
1075      1
50        1
38577     1
39        1
2459      1
12940     1
11        1
2         1
Name: nans, dtype: int64

### 54 columns have 39717 'nans' - same as the number of rows. Drop these columns.

In [39]:
df = df.dropna(axis=1, how='all')
df.shape

(39717, 57)

## Columns with same values - these are not useful for analysis

In [40]:
same_columns = {}
for c in df.columns:
    if df[c].value_counts().shape[0] == 1:
        same_columns[c] = list(df[c].unique())
same_columns

{'pymnt_plan': ['n'],
 'initial_list_status': ['f'],
 'collections_12_mths_ex_med': [0.0, nan],
 'policy_code': [1],
 'application_type': ['INDIVIDUAL'],
 'acc_now_delinq': [0],
 'chargeoff_within_12_mths': [0.0, nan],
 'delinq_amnt': [0],
 'tax_liens': [0.0, nan]}

### Drop columns with same values

In [41]:
df = df.drop(same_columns.keys(), axis=1)
df.shape

(39717, 48)

In [42]:
# Check null values again
df[df.columns[df.isnull().any()]].isnull().sum()

emp_title                  2459
emp_length                 1075
desc                      12940
title                        11
mths_since_last_delinq    25682
mths_since_last_record    36931
revol_util                   50
last_pymnt_d                 71
next_pymnt_d              38577
last_credit_pull_d            2
pub_rec_bankruptcies        697
dtype: int64

### Columns with more than 50% missing values

In [43]:
too_many_nans = df[df.columns[df.isnull().sum() * 100 / df.shape[0] > 50]].columns.tolist()
too_many_nans

['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d']

In [44]:
df = df.drop(too_many_nans, axis=1)
df.shape

(39717, 45)

In [45]:
df[df.columns[df.isnull().any()]].isnull().sum()

emp_title                2459
emp_length               1075
desc                    12940
title                      11
revol_util                 50
last_pymnt_d               71
last_credit_pull_d          2
pub_rec_bankruptcies      697
dtype: int64

### Check columns one by one

In [46]:
df.desc.describe()

count     26777
unique    26527
top            
freq        210
Name: desc, dtype: object

In [47]:
df.desc.head()

0      Borrower added on 12/22/11 > I need to upgra...
1      Borrower added on 12/22/11 > I plan to use t...
2                                                  NaN
3      Borrower added on 12/21/11 > to pay for prop...
4      Borrower added on 12/21/11 > I plan on combi...
Name: desc, dtype: object

In [27]:
df.emp_title.value_counts()

US Army                       134
Bank of America               109
IBM                            66
AT&T                           59
Kaiser Permanente              56
                             ... 
Hartz Mountain Corportaion      1
Bell Ford                       1
Cooking Matters                 1
columbia university             1
County Of Ocean                 1
Name: emp_title, Length: 28820, dtype: int64

#### Can drop 'emp_title' and 'desc' - they seem to have too many values to analyze for now

In [30]:
df.emp_length.value_counts()

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
Name: emp_length, dtype: int64

In [48]:
# Convert emp_length to integer
import re
df.emp_length = df.emp_length.fillna('unknown')
df.emp_length.value_counts()

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
unknown      1075
Name: emp_length, dtype: int64

In [53]:
df.title.value_counts()

Debt Consolidation                    2184
Debt Consolidation Loan               1729
Personal Loan                          659
Consolidation                          517
debt consolidation                     505
                                      ... 
Antonio's Tacos                          1
Consolidating Debt, after job loss       1
credit card re-fi                        1
Major Investment                         1
Payoff american express                  1
Name: title, Length: 19615, dtype: int64

In [54]:
len(df.title.unique().tolist())

19616

In [55]:
df.title = df.title.fillna('unknown')
len(df.title.unique().tolist())

19616

#### Can drop 'title' 
It has 19k unique values.  
Ideally it may be possible to consolidate these, into broad categories like 'debt consolodation', 'credit card' etc.  
Skipping for now.

In [58]:
df = df.drop(['title', 'desc', 'emp_title'], axis=1)
df.shape

(39717, 42)

In [59]:
df.revol_util.value_counts()

0%        977
0.20%      63
63%        62
40.70%     58
66.70%     58
         ... 
17.78%      1
24.65%      1
0.12%       1
24.63%      1
25.33%      1
Name: revol_util, Length: 1089, dtype: int64

In [60]:
df.last_pymnt_d.value_counts()

May-16    1256
Mar-13    1026
Dec-14     945
May-13     907
Feb-13     869
          ... 
Nov-08      10
Jun-08      10
Mar-08       5
Jan-08       4
Feb-08       1
Name: last_pymnt_d, Length: 101, dtype: int64

In [65]:
# dropping nans for 'last_pymnt_d', 'revol_util', 'last_credit_pull_d'
# We loose very little data
df1 = df[df['last_pymnt_d'].notna()]
df1 = df1[df1['revol_util'].notna()]
df1 = df1[df1['last_credit_pull_d'].notna()]
100 - df1.shape[0] / df.shape[0] * 100

0.29961981015685524

#### final data lost is only 0.30%

In [68]:
df1.pub_rec_bankruptcies.value_counts()

0.0    37228
1.0     1667
2.0        7
Name: pub_rec_bankruptcies, dtype: int64

In [70]:
df1.pub_rec_bankruptcies.dtype

dtype('float64')

#### Fill nans with mode 0.0

In [74]:
df1.pub_rec_bankruptcies = df1.pub_rec_bankruptcies.fillna(df1.pub_rec_bankruptcies.mode(dropna=True)[0])

In [75]:
df1[df1.columns[df1.isnull().any()]].isnull().sum()

Series([], dtype: float64)

## This Ends handling all missing values in the data
---

In [81]:
# check remaining columns
df1.dtypes

id                           int64
member_id                    int64
loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
url                         object
purpose                     object
zip_code                    object
addr_state                  object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec                      int64
revol_bal                    int64
revol_util                  object
total_acc           

In [82]:
# drop un-necessary columns
# url - only a link. Will not be useful
# recoveries - we want to see which loan will default - so anything after default is not useful
# id, member_id - only ids, not useful
# collection_recovery_fee - post charge-off 
cols_to_drop = ['id', 'member_id', 'url', 'recoveries', 'collection_recovery_fee']
df1 = df1.drop(cols_to_drop, axis=1)
df1.shape

(39598, 37)

In [78]:
df1.loan_status.value_counts()

Fully Paid     32915
Charged Off     5543
Current         1140
Name: loan_status, dtype: int64

In [9]:
df.loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64